# Bonus task - Pipeline of ML for solving PDE

The purpose of this task is to create a pipeline of machine learning models to solve a PDE. There are three main steps to follow:
1. Implementing stochastic samplers of discretised functions from three different classes of functions: Gaussian processes (GP), piecewise-linear functions (PL) and Chebyshev polynomials (CP)
2. Employ a finite difference solver for the Poisson equation for various forcing terms.
3. Performing zero-shot learning and transfer learning or fine-tuning to explore the generalization properties of your models.

## Step 0: Importing libraries

In [ ]:
# libraries for step 1
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import numpy.polynomial.chebyshev as cheb

# libraries for step 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# libraries for step 3
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.nn as nn
import torch.optim as optim

## Step 1: Implementing stochastic samplers

In order to implement the stochastic samplers there are three main classes of functions to consider: Gaussian processes (GP), piecewise-linear functions (PL) and Chebyshev polynomials (CP).
It's important to set up a sample_function and a function to solve_bvp. After that, data can be generated and the models can be trained.

In [ ]:
def sample_function(class_type, x_grid):
    if class_type == 'GP':
        # Gaussian Process with RBF kernel
        kernel = RBF(length_scale=1.0)
        gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1)
        f_x = gp.sample_y(x_grid.reshape(-1, 1), 1).flatten()
    elif class_type == 'PL':
        # Piecewise Linear Function
        num_pieces = np.random.randint(2, 10)  # Choose a random number of pieces. This line generates a random integer between 2 and 9 (inclusive). The code is set up in this way given the possibility of adding richness to the model by increasing the number of pieces and variability.
        breakpoints = np.sort(np.random.choice(x_grid, num_pieces, replace=False))
        slopes = np.random.randn(num_pieces)
        intercepts = np.random.randn(num_pieces)

        f_x = np.zeros_like(x_grid)
        for i in range(1, len(breakpoints)):
            mask = (x_grid >= breakpoints[i-1]) & (x_grid <= breakpoints[i])
            # This creates a boolean mask for the x_grid array. The mask is True for all x values that lie within the current segment of the piecewise linear function.
            f_x[mask] = slopes[i-1] * x_grid[mask] + intercepts[i-1]
            # This line computes the value of the piecewise linear function for the x values that lie within the current segment. The value is computed using the equation of a line.
    elif class_type == 'CP':
        # Chebyshev Polynomials
        num_polynomials = np.random.randint(1, 7)
        coeffs = np.random.randn(num_polynomials) # Random coefficients for the Chebyshev polynomials that depend on the number of polynomials
        cheb_polys = [cheb.Chebyshev.basis(i) for i in range(num_polynomials)] # this function creates the Chebyshev polynomials up to the number of polynomials. The initial Chebyshev polynomial is the zeroth order Chebyshev polynomial, which is a constant function equal to 1. The first order Chebyshev polynomial is a linear function. The second order Chebyshev polynomial is a quadratic function, and so on.

        f_x = sum(c * p(x_grid) for c, p in zip(coeffs, cheb_polys))
        # This line computes the value of the Chebyshev polynomial function for the x values in x_grid. The value is computed as the sum of the product of the coefficients and the Chebyshev polynomials evaluated at the x values.
    return f_x

def solve_bvp(f_x, x_grid):
    # solve_bvp has the purpose of solving the Poisson equation for various forcing terms
    n = len(x_grid)
    h = x_grid[1] - x_grid[0]

    # Create the second derivative matrix
    A = -2 * np.eye(n) + np.eye(n, k=1) + np.eye(n, k=-1)
    A /= h**2

    # Apply boundary conditions
    A[0, 0] = 1
    A[0, 1] = 0
    A[-1, -1] = 1
    A[-1, -2] = 0

    f_x[0] = 0
    f_x[-1] = 0

    # Solve the linear system
    u_x = np.linalg.solve(A, f_x)

    return u_x

# Initialize dataset
D = []

x_grid = np.linspace(-1, 1, 100)

for class_type in ['GP', 'PL', 'CP']:
    for i in range(100):
        f_x = sample_function(class_type, x_grid)
        u_x = solve_bvp(f_x, x_grid)
        D.append((f_x, u_x))

# Output the dataset D
# print(D)


[(array([ 0.        , -2.06945021, -2.03991421, -2.01099382, -1.98273939,
       -1.95520081, -1.92842269, -1.90244721, -1.87731282, -1.85305494,
       -1.82970473, -1.80729057, -1.78583626, -1.76536214, -1.74588466,
       -1.72741641, -1.70996617, -1.69353928, -1.6781364 , -1.66375556,
       -1.65039061, -1.63803155, -1.62666525, -1.61627488, -1.60684091,
       -1.5983398 , -1.59074583, -1.58402996, -1.57816055, -1.57310351,
       -1.56882227, -1.56527818, -1.56243034, -1.56023652, -1.5586528 ,
       -1.55763331, -1.55713174, -1.5571001 , -1.55749012, -1.55825284,
       -1.55933886, -1.56069818, -1.56228141, -1.56403901, -1.56592203,
       -1.56788186, -1.56987037, -1.57184111, -1.57374741, -1.57554501,
       -1.57719032, -1.57864097, -1.57985656, -1.58079841, -1.58142904,
       -1.58171327, -1.58161773, -1.58111065, -1.58016282, -1.57874662,
       -1.57683677, -1.57441017, -1.57144524, -1.56792348, -1.56382747,
       -1.55914264, -1.55385595, -1.54795643, -1.54143554, -1.

here it's possible to see the output of the dataset D.

In [ ]:
len(D)

300

In this case we obtain a dataset with 300 elements, each element is a tuple containing the function f_x and the solution u_x of the Poisson equation for the given function. The dataset is generated by sampling functions from three different classes: Gaussian processes (GP), piecewise-linear functions (PL), and Chebyshev polynomials (CP). Each class has 100 samples.

In [ ]:
# We can now split the dataset into three separate datasets for each class. The first 100 samples are from the GP class, the next 100 samples are from the PL class, and the last 100 samples are from the CP class.
DGP = D[:100]
DPL = D[100:200]
DCP = D[200:]

## Step 2: Training the models

The first activity is to set up the ML architecture. We are going to use a Neural Operator. The Neural Operator is a neural network that learns to approximate the solution of a PDE. The Neural Operator is trained to approximate the solution of the PDE by minimizing the mean squared error between the predicted solution and the true solution. The Neural Operator is trained using the finite difference solver for the Poisson equation.

In [ ]:
# Define the Neural Operator architecture
class NeuralOperator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralOperator, self).__init__()
        # Define your neural network architecture here
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Forward pass of the neural network
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Function to train a Neural Operator for a given class
def train_neural_operator(class_type, dataset):
    # Prepare data for training
    x_train, y_train = zip(*dataset)

    # It's useful to consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor.
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    # Convert NumPy arrays to PyTorch tensors
    x_train = torch.FloatTensor(x_train)
    y_train = torch.FloatTensor(y_train)

    # Define hyperparameters
    input_dim = x_train.shape[1]  # Dimension of input data
    output_dim = y_train.shape[1]  # Dimension of output data
    hidden_dim = 64  # Dimension of hidden layers
    batch_size = 32
    learning_rate = 0.001
    num_epochs = 10

    # Initialize Neural Operator
    neural_operator = NeuralOperator(input_dim, hidden_dim, output_dim)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(neural_operator.parameters(), lr=learning_rate)

    # Create DataLoader for batching
    train_data = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Train the Neural Operator
    for epoch in range(num_epochs):
        neural_operator.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = neural_operator(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    return neural_operator


In [ ]:
# Initialize results storage R
R = {}

# Dictionary to map class names to dataset variables
dataset_dict = {'GP': DGP, 'PL': DPL, 'CP': DCP}

# Train one Neural Operator per class
trained_operators = {}
for class_type, dataset in dataset_dict.items():
    print(f"Training Neural Operator for class: {class_type}")
    trained_operator = train_neural_operator(class_type, dataset)
    trained_operators[class_type] = trained_operator

# Output: Trained operators {NGP, NPL, NCP}
print("Trained operators:", trained_operators)

Training Neural Operator for class: GP
Epoch [1/10], Loss: 0.3777
Epoch [2/10], Loss: 0.2561
Epoch [3/10], Loss: 0.1941
Epoch [4/10], Loss: 0.1466
Epoch [5/10], Loss: 0.1059
Epoch [6/10], Loss: 0.0734
Epoch [7/10], Loss: 0.0496
Epoch [8/10], Loss: 0.0325
Epoch [9/10], Loss: 0.0207
Epoch [10/10], Loss: 0.0131
Training Neural Operator for class: PL
Epoch [1/10], Loss: 0.0912
Epoch [2/10], Loss: 0.0627
Epoch [3/10], Loss: 0.0477
Epoch [4/10], Loss: 0.0370
Epoch [5/10], Loss: 0.0289
Epoch [6/10], Loss: 0.0219
Epoch [7/10], Loss: 0.0167
Epoch [8/10], Loss: 0.0127
Epoch [9/10], Loss: 0.0100
Epoch [10/10], Loss: 0.0084
Training Neural Operator for class: CP
Epoch [1/10], Loss: 0.2665
Epoch [2/10], Loss: 0.1688
Epoch [3/10], Loss: 0.1249
Epoch [4/10], Loss: 0.0889
Epoch [5/10], Loss: 0.0612
Epoch [6/10], Loss: 0.0392
Epoch [7/10], Loss: 0.0240
Epoch [8/10], Loss: 0.0168
Epoch [9/10], Loss: 0.0128
Epoch [10/10], Loss: 0.0106
Trained operators: {'GP': NeuralOperator(
  (fc1): Linear(in_features=

# Step 3: Performing zero-shot learning and transfer learning or fine-tuning

In this step, we are going to evaluate the generalization properties of the trained Neural Operators using zero-shot learning and transfer learning or fine-tuning. We will evaluate the performance of the trained Neural Operators on a test dataset that was not seen during training.

In [ ]:
# Algorithm 3: Fine-tune Operators on Other Classes

# Define the neural operator model class (already defined earlier)
class NeuralOperator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralOperator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

# Helper functions
def evaluate_performance(model, dataloader):
    criterion = nn.MSELoss()
    total_loss = 0.0
    count = 0

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for f_batch, u_batch in dataloader:
            f_batch, u_batch = f_batch.float(), u_batch.float()  # Ensure correct dtype
            outputs = model(f_batch)
            loss = criterion(outputs, u_batch)
            total_loss += loss.item() * f_batch.size(0)
            count += f_batch.size(0)

    model.train()  # Set model back to training mode
    return total_loss / count

def sample_fine_tune_data(dataset, num_samples=20):
    indices = np.random.choice(len(dataset), num_samples, replace=False)
    fine_tune_subset = Subset(dataset, indices)
    fine_tune_dataloader = DataLoader(fine_tune_subset, batch_size=num_samples, shuffle=True)
    return fine_tune_dataloader

def fine_tune_operator(model, fine_tune_dataloader, num_epochs=20, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for f_batch, u_batch in fine_tune_dataloader:
            f_batch, u_batch = f_batch.float(), u_batch.float()  # Ensure correct dtype
            optimizer.zero_grad()
            outputs = model(f_batch)
            loss = criterion(outputs, u_batch)
            loss.backward()
            optimizer.step()

    return model

def compute_relative_l2_distance(zero_shot_performance, fine_tuned_performance):
    return (zero_shot_performance - fine_tuned_performance) / zero_shot_performance

# Algorithm 3: Fine-tune Operators on Other Classes
def fine_tune_operators(DGP, DPL, DCH, trained_operators):
    # Initialize results storage
    R = {}

    datasets = {
        'GP': DGP,
        'PL': DPL,
        'CP': DCP
    }

    for train_class in ['GP', 'PL', 'CP']:
        eval_classes = ['GP', 'PL', 'CP']
        eval_classes.remove(train_class)

        for eval_class in eval_classes:
            print(f"Fine-tuning {train_class} operator on {eval_class} class")

            # Prepare evaluation data loader
            D_eval_class = datasets[eval_class]
            eval_dataloader = DataLoader(D_eval_class, batch_size=32, shuffle=False)

            # Evaluate zero-shot performance
            zero_shot_performance = evaluate_performance(trained_operators[train_class], eval_dataloader)

            # Sample 20 examples for fine-tuning
            fine_tune_dataloader = sample_fine_tune_data(D_eval_class, num_samples=20)

            # Fine-tune the operator on the fine-tune data
            fine_tuned_operator = fine_tune_operator(trained_operators[train_class], fine_tune_dataloader, num_epochs=20)

            # Evaluate fine-tuned performance
            fine_tuned_performance = evaluate_performance(fine_tuned_operator, eval_dataloader)

            # Compute relative L2 distance
            relative_l2_distance = compute_relative_l2_distance(zero_shot_performance, fine_tuned_performance)

            # Store results in R
            R[(train_class, eval_class)] = relative_l2_distance

    # Output fine-tuned results
    return R

# Assuming `trained_operators` contains the trained models for GP, PL, and CP
fine_tune_results = fine_tune_operators(DGP, DPL, DCP, trained_operators)

# Print the fine-tuning results
import pprint
pprint.pprint(fine_tune_results)


Fine-tuning GP operator on PL class
Fine-tuning GP operator on CP class
Fine-tuning PL operator on GP class
Fine-tuning PL operator on CP class
Fine-tuning CP operator on GP class
Fine-tuning CP operator on PL class
{('CP', 'GP'): 0.946042644541795,
 ('CP', 'PL'): 0.3221547371759305,
 ('GP', 'CP'): 0.7871888928320445,
 ('GP', 'PL'): 0.23226263780196385,
 ('PL', 'CP'): 0.4698056582451354,
 ('PL', 'GP'): 0.6166607317329166}


We can observe the following results

| Train Class | Eval Class | Relative L2 Distance |
|-------------|------------|----------------------|
| CP          | GP         | 0.9460               |
| CP          | PL         | 0.3222               |
| GP          | CP         | 0.7872               |
| GP          | PL         | 0.2323               |
| PL          | CP         | 0.4698               |
| PL          | GP         | 0.6167               |

what is important to notice is that a higher relative L2 distance means that the fine-tuned model's performance is significantly better than its zero-shot performance. This could suggest that the model has effectively adapted to the characteristics of the new class during fine-tuning.
Conversely, a lower relative L2 distance would indicate that the fine-tuned model's performance is closer to its zero-shot performance, implying less effective adaptation or improvement after fine-tuning.


# Conclusion

## Does training on any one specific class offer (on average) superior zero-shot performance on the others?

Based on the provided relative L2 distances.
We could divide the comparison between the models.

- **CP (Chebyshev Polynomials) vs. GP (Gaussian Processes) and PL (Piecewise-linear Functions)**:
  - Models trained on CP show a higher relative improvement when evaluated on GP (0.9460) compared to PL (0.3222).

- **GP (Gaussian Processes) vs. CP and PL**:
  - Models trained on GP also show notable relative improvements when evaluated on CP (0.7872) compared to PL (0.2323).

- **PL (Piecewise-linear Functions) vs. CP and GP**:
  - Models trained on PL exhibit moderate relative improvements when evaluated on CP (0.4698) and GP (0.6167).

### Reasons for Differences in Zero-Shot Performance Across Classes

This suggests potential structural similarities or easier adaptation from CP to GP.
This may indicate beneficial properties of GP that aid generalization to CP.
This indicates some level of shared characteristics with both, though perhaps not as advantageous as CP to GP or vice versa.

1. **Structural Similarities**: It's possible that a higher relative L2 distance may be given to some structure similaries. For example, CP and GP tend to exhibit smooth and regular behavior due to their mathematical constructions. On the opposite side, PL functions are locally linear, which means they can approximate more complex functions by combining linear segments. This characteristic contrasts with the global smoothness of CP and GP.

2. **Training Dynamics**: The training process can bias models towardslearning certain types of functions more effectively, influencing how well they generalize to unseen classes during zero-shot evaluation. For example, Mean Squared Error (MSE) loss emphasizes minimizing the squared differences between predicted and actual values, which might bias the model towards learning functions that minimize this specific type of error. Moreover, other techniques such as L2 regularization, dropout can influence the final result.

3. **Model Capacity and Representation**: Neural networks may have varying capacities to represent different function types. Neural networks with larger capacities (more parameters, deeper architectures) can represent complex functions with intricate relationships and nonlinearities.

## Are PINNs a suitable choice for solving this task?

Physics-Informed Neural Networks (PINNs) are a suitable choice for solving tasks like solving the Poisson equation, especially when the underlying physics is known or can be represented mathematically like in this case of Poisson equation.
However, the suitability of PINNs depends on various factors such as the complexity of the problem, availability of data, and computational resources.
In this task, where the goal is to solve the Poisson equation for different types of function spaces, PINNs could have several advantages:

1. **Combination of Physics and Data-Driven Learning**:
   - PINNs integrate domain knowledge (physics principles encoded in PDEs) with data-driven learning using neural networks.
   - For the Poisson equation (\(-\Delta u = f(x)\)), PINNs can directly enforce the PDE constraints, ensuring physically consistent solutions.

2. **Efficient Data Utilization**:
   - PINNs can generalize from relatively small datasets by leveraging the physics constraints embedded in PDEs.
   - They do not require large amounts of labeled data, making them suitable for scenarios where data generation is based on stochastic samplers.

3. **Zero-Shot Learning and Generalization**:
   - PINNs can generalize to new function classes (e.g., from GP to PL or CP) by leveraging learned physics principles and neural network approximation capabilities.
   - Techniques like fine-tuning and transfer learning can further enhance their adaptability to new classes.
